In [ ]:
# -*- coding: utf-8 -*-
"""
Example script to reconstruct the SPECT simulation. This is almost the equivalent of
    ./run_reconstruction_SPECT.sh

However, that scripts runs FBP and MAP reconstructions as well. This file does not.

Author: Kris Thielemans
"""

# Initial imports

In [ ]:
import stir
from stirextra import *
import os
from distutils.dir_util import copy_tree
import matplotlib.pyplot as plt

# go to directory with input files

In [ ]:
# adapt this path to your situation (or start everything in the exercises directory)
os.chdir(os.getenv('STIR_exercises_PATH'))

# copy.par files and make directory for output files

In [ ]:
copy_tree('EX_reconstruction_SPECT/', 'working_folder/single_slice_SPECT')
os.chdir('working_folder/single_slice_SPECT')

# read in data

In [ ]:
emission=stir.FloatVoxelsOnCartesianGrid.read_from_file('emission.hv')
atn=stir.FloatVoxelsOnCartesianGrid.read_from_file('CTAC.hv')
template=stir.ProjData.read_from_file('template_sinogram.hs')
proj_data=stir.ProjData.read_from_file('my_sim.hs')

# reconstruction with OSEM using an existing parameter file

STIR reconstructions can be initialised via a `.par` file, which is just a text file containing all the settings. We will do that here first.

In [ ]:
recon = stir.OSMAPOSLReconstruction3DFloat('OSEM.par')

In [ ]:
recon.set_num_subsets(8)
recon.set_num_subiterations(10)
recon.set_save_interval(5)

We can then modify the parameters from Python. For example setting the data that will be reconstructed:

In [ ]:
recon.set_input_data(proj_data)

Read in an image as initialisation (although in this example we change its values then, but at least the number of voxels etc will be kept).

In [ ]:
target = stir.FloatVoxelsOnCartesianGrid.read_from_file('init.hv')
# we will just fill the whole array with 1 here
target.fill(1)

Set-up the reconstruction object (needs to be done once)

In [ ]:
s = recon.set_up(target)
if (s == stir.Succeeded(stir.Succeeded.no)):
    raise RuntimeError("recon set_up problem")

Run the reconstruction! The line below will update the `target` object.

In [ ]:
recon.reconstruct(target);

Display

In [ ]:
npimage = to_numpy(target)
plt.imshow(npimage[0,:,:])

# Reconstruction set-up in Python

## Create projection matrix and projector

It is also possible to set-up reconstruction parameters directly in Python. Note that some parameters, including setting the projectors, need STIR 5.2. However, if you don't have that yet, you can first use the `.par` file, and then change settings from Python.

In [ ]:
pm=stir.ProjMatrixByBinSPECTUB()

In [ ]:
pm.set_attenuation_image_sptr(atn)

In [ ]:
projectors=stir.ProjectorByBinPairUsingProjMatrixByBin(pm)
projectors.set_up(template.get_proj_data_info(), emission)

## Create reconstruction (and its objective function)

In [ ]:
recon = stir.OSMAPOSLReconstruction3DFloat()
obj_fun = recon.get_objective_function()
# cannot do this yet
# obj_fun.set_projector_pair(projectors)
recon.set_input_data(proj_data)
recon.set_num_subsets(8)
recon.set_num_subiterations(10)

In [ ]:
help(obj_fun)

Now you can reconstruct as above.